In [102]:
import os
import pandas as pd

base_dir = "../../Data"
base_csv_path = os.path.join(base_dir, 'm0_imputed.csv')
output_csv_path = os.path.join(base_dir, 'm0_post_feat_sel.csv')
assert os.path.exists(base_csv_path), f"base {base_csv_path} does not exist"

df = pd.read_csv(base_csv_path)

In [103]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression

mean_wl_time = df['wl_time'].mean()
print(f"Mean of wl_time: {mean_wl_time}")

X = df.drop(['wl_time'], axis=1)
y = df['wl_time'].apply(lambda x: 1 if x > mean_wl_time else 0)

Mean of wl_time: 181.90769230769232


In [104]:
from sklearn.preprocessing import StandardScaler

columns_to_scale = [col for col in X.columns if X[col].nunique() > 2]
scaler = StandardScaler()
X[columns_to_scale] = scaler.fit_transform(X[columns_to_scale])

In [105]:
log = LogisticRegression(max_iter=1000)

selector = RFECV(estimator=log, cv=5, scoring='f1', verbose=1)
selector.fit(X, y)

print(f"Num of features: {X.shape[1]}")
print(f"Num of features recommended after feature selection: {selector.n_features_}")

Fitting estimator with 113 features.
Fitting estimator with 112 features.
Fitting estimator with 111 features.
Fitting estimator with 110 features.
Fitting estimator with 109 features.
Fitting estimator with 108 features.
Fitting estimator with 107 features.
Fitting estimator with 106 features.
Fitting estimator with 105 features.
Fitting estimator with 104 features.
Fitting estimator with 103 features.
Fitting estimator with 102 features.
Fitting estimator with 101 features.
Fitting estimator with 100 features.
Fitting estimator with 99 features.
Fitting estimator with 98 features.
Fitting estimator with 97 features.
Fitting estimator with 96 features.
Fitting estimator with 95 features.
Fitting estimator with 94 features.
Fitting estimator with 93 features.
Fitting estimator with 92 features.
Fitting estimator with 91 features.
Fitting estimator with 90 features.
Fitting estimator with 89 features.
Fitting estimator with 88 features.
Fitting estimator with 87 features.
Fitting estima

In [106]:
feature_rankings = list(zip(df.columns, selector.ranking_, selector.support_))
feature_rankings_sorted = sorted(feature_rankings, key=lambda x: x[1])

X_new = df[[col for col in df.columns if col in X.columns[selector.support_]]]
df_sel = pd.concat([X_new, y], axis=1)

log.fit(X_new, y)  # Fit to get weight coefficients

for feature, ranking, support in feature_rankings_sorted:
    print(f"Feature: {feature}, Ranking: {ranking}, Support: {support}")

Feature: wgt_kg_tcr, Ranking: 1, Support: True, Weight: 0.0373926325648934
Feature: func_stat_tcr, Ranking: 1, Support: True, Weight: -0.0006729224289989613
Feature: hemo_co_tcr, Ranking: 1, Support: True, Weight: -0.3061935757556832
Feature: init_stat, Ranking: 1, Support: True, Weight: 0.0009228904372331337
Feature: init_age, Ranking: 1, Support: True, Weight: 0.007736511373703416
Feature: num_prev_tx_10, Ranking: 1, Support: True, Weight: 0
Feature: tah_N, Ranking: 1, Support: True, Weight: -0.46418737643219987
Feature: vas_N, Ranking: 1, Support: True, Weight: 0
Feature: vas_missing, Ranking: 1, Support: True, Weight: 0
Feature: onvent_missing, Ranking: 1, Support: True, Weight: 0
Feature: icu_missing, Ranking: 1, Support: True, Weight: 0
Feature: inotropic_N, Ranking: 1, Support: True, Weight: 0.7705452842872637
Feature: inotropic_Y, Ranking: 1, Support: True, Weight: -0.43339344949258496
Feature: inotropic_missing, Ranking: 1, Support: True, Weight: -0.8855745952398255
Feature: g

In [ ]:
#Print the weights of the model with the corresponding feature name
weights = list(zip(X_new.columns, log.coef_[0]))
weights_sorted = sorted(weights, key=lambda x: x[1])
for feature, weight in weights_sorted:
    print(f"Feature: {feature}, Weight: {weight}")

In [ ]:
df_sel.to_csv(output_csv_path, index=False)